# STDP Curve (Hard)
This task will attempt to plot an STDP curve, showing how the weight change varies with timing between
spikes.

Set up the simulation to use a 1ms time step.

In [ ]:
import spynnaker.pyNN as p
p.setup(1.0)

Create a population of 100 presynaptic neurons.

In [ ]:
pre = p.Population(100, p.IF_curr_exp())

Create a spike source array population of 100 sources connected to the presynaptic population. Set
the spikes in the arrays so that each spikes twice 200ms apart, and that the first spike for each is
1ms after the first spike of the last e.g. [[0, 200], [1, 201], …] (hint: you can do this with a list
comprehension).  Note the second spike is to make sure that the STDP computations take place with the deferred execution model.

In [ ]:
pre_stim = p.Population(100, p.SpikeSourceArray([[i, 200 + i] for i in range(100)]))
p.Projection(pre_stim, pre, p.OneToOneConnector(), p.StaticSynapse(weight=5.0))

Create a population of 100 postsynaptic neurons.

In [ ]:
post = p.Population(100, p.IF_curr_exp())

Create a spike source array connected to the postsynaptic neurons all spiking at 50ms.

In [ ]:
post_stim = p.Population(1, p.SpikeSourceArray([50]))
p.Projection(post_stim, post, p.AllToAllConnector(), p.StaticSynapse(weight=5.0))

Record the spikes from the presynaptic and postsynaptic populations.

In [ ]:
pre.record("spikes")
post.record("spikes")

Connect the presynaptic population to the postsynaptic population with an STDP projection with an
initial weight of 0.5 and a maximum of 1 and minimum of 0, and a tau_plus and tau_minus of 20.0 and an A_plus and A_minus of 0.1

In [ ]:
timing = p.SpikePairRule(tau_plus=20.0, tau_minus=20.0, A_plus=0.1, A_minus=0.1)
weight = p.AdditiveWeightDependence(w_min=0.0, w_max=1.0)
proj = p.Projection(pre, post, p.OneToOneConnector(), p.STDPMechanism(timing_dependence=timing, weight_dependence=weight, weight=0.5))

Run the simulation for 500ms.

In [ ]:
p.run(500)

Get the weights and the presynaptic and postsynaptic spikes.

In [ ]:
import numpy
weights = numpy.array(proj.get("weight", "list", with_address=False))
pre_spikes = pre.get_data("spikes").segments[0].spiketrains
post_spikes = post.get_data("spikes").segments[0].spiketrains
p.end()

Draw a graph of the weight changes from the initial weight value against the difference in
presynaptic and postsynaptic neurons (hint: the presynaptic neurons should spike twice but the
postsynaptic should only spike once; you are looking for the first spike from each presynaptic
neuron).

In [ ]:
weight_diff = weights - 0.5
time_diff = [pre_neuron_spikes[0] - post_neuron_spikes[0] for pre_neuron_spikes, post_neuron_spikes in zip(pre_spikes, post_spikes)]

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(time_diff, weight_diff)
plt.show()